In [1]:
%pip install tiktoken
%pip install python-dotenv
%pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tiktoken
import os
import json
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

We are going to test multiple prompts by estimating the tokens prior to sending them to the LLM. Then, we will get the actual tokens, and thereafter, we can find the percentage difference to see how far off our estimate is."

In [3]:
prompts = {
    1: '''
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/model/InventoryEntity.java"
Source file contents:
```java
package com.redhat.coolstore.model;

import java.io.Serializable;

import javax.persistence.Column;
import javax.persistence.Entity;
import javax.persistence.Id;
import javax.persistence.Table;
import javax.persistence.UniqueConstraint;
import javax.xml.bind.annotation.XmlRootElement;

@Entity
@XmlRootElement
@Table(name = "INVENTORY", uniqueConstraints = @UniqueConstraint(columnNames = "itemId"))
public class InventoryEntity implements Serializable {

	private static final long serialVersionUID = 7526472295622776147L; 

    @Id
    private String itemId;


    @Column
    private String location;


    @Column
    private int quantity;


    @Column
    private String link;

    public InventoryEntity() {

    }

    public String getItemId() {
		return itemId;
	}

	public void setItemId(String itemId) {
		this.itemId = itemId;
	}

	public String getLocation() {
		return location;
	}

	public void setLocation(String location) {
		this.location = location;
	}

	public int getQuantity() {
		return quantity;
	}

	public void setQuantity(int quantity) {
		this.quantity = quantity;
	}

	public String getLink() {
		return link;
	}

	public void setLink(String link) {
		this.link = link;
	}

	@Override
    public String toString() {
        return "InventoryEntity [itemId=" + itemId + ", availability=" + quantity + "/" + location + " link=" + link + "]";
    }
}

```

## Issues

### incident 0
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 5
### incident 1
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 6
### incident 2
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 7
### incident 3
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 8
### incident 4
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 9
### incident 5
incident to fix: "Replace the `javax.xml` import statement with `jakarta.xml`"
Line number: 10

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.

''',
    2: '''
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/service/InventoryNotificationMDB.java"
Source file contents:
```java
package com.redhat.coolstore.service;

import com.redhat.coolstore.model.Order;
import com.redhat.coolstore.utils.Transformers;

import javax.inject.Inject;
import javax.jms.*;
import javax.naming.Context;
import javax.naming.InitialContext;
import javax.naming.NamingException;
import javax.rmi.PortableRemoteObject;
import java.util.Hashtable;

public class InventoryNotificationMDB implements MessageListener {

    private static final int LOW_THRESHOLD = 50;

    @Inject
    private CatalogService catalogService;

    private final static String JNDI_FACTORY = "weblogic.jndi.WLInitialContextFactory";
    private final static String JMS_FACTORY = "TCF";
    private final static String TOPIC = "topic/orders";
    private TopicConnection tcon;
    private TopicSession tsession;
    private TopicSubscriber tsubscriber;

    public void onMessage(Message rcvMessage) {
        TextMessage msg;
        {
            try {
                System.out.println("received message inventory");
                if (rcvMessage instanceof TextMessage) {
                    msg = (TextMessage) rcvMessage;
                    String orderStr = msg.getBody(String.class);
                    Order order = Transformers.jsonToOrder(orderStr);
                    order.getItemList().forEach(orderItem -> {
                        int old_quantity = catalogService.getCatalogItemById(orderItem.getProductId()).getInventory().getQuantity();
                        int new_quantity = old_quantity - orderItem.getQuantity();
                        if (new_quantity < LOW_THRESHOLD) {
                            System.out.println("Inventory for item " + orderItem.getProductId() + " is below threshold (" + LOW_THRESHOLD + "), contact supplier!");
                        } else {
                            orderItem.setQuantity(new_quantity);
                        }
                    });
                }


            } catch (JMSException jmse) {
                System.err.println("An exception occurred: " + jmse.getMessage());
            }
        }
    }

    public void init() throws NamingException, JMSException {
        Context ctx = getInitialContext();
        TopicConnectionFactory tconFactory = (TopicConnectionFactory) PortableRemoteObject.narrow(ctx.lookup(JMS_FACTORY), TopicConnectionFactory.class);
        tcon = tconFactory.createTopicConnection();
        tsession = tcon.createTopicSession(false, Session.AUTO_ACKNOWLEDGE);
        Topic topic = (Topic) PortableRemoteObject.narrow(ctx.lookup(TOPIC), Topic.class);
        tsubscriber = tsession.createSubscriber(topic);
        tsubscriber.setMessageListener(this);
        tcon.start();
    }

    public void close() throws JMSException {
        tsubscriber.close();
        tsession.close();
        tcon.close();
    }

    private static InitialContext getInitialContext() throws NamingException {
        Hashtable<String, String> env = new Hashtable<>();
        env.put(Context.INITIAL_CONTEXT_FACTORY, JNDI_FACTORY);
        env.put(Context.PROVIDER_URL, "t3://localhost:7001");
        env.put("weblogic.jndi.createIntermediateContexts", "true");
        return new InitialContext(env);
    }
}
```

## Issues

### incident 0
incident to fix: "Replace the `javax.inject` import statement with `jakarta.inject`"
Line number: 6
### incident 1
incident to fix: "References to JavaEE/JakartaEE JMS elements should be removed and replaced with their Quarkus SmallRye/Microprofile equivalents."
Line number: 7

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.

''',
    3: '''
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/model/OrderItem.java"
Source file contents:
```java
package com.redhat.coolstore.model;

import java.io.Serializable;

import javax.persistence.Column;
import javax.persistence.Entity;
import javax.persistence.GeneratedValue;
import javax.persistence.Id;
import javax.persistence.Table;

@Entity
@Table(name = "ORDER_ITEMS")
public class OrderItem implements Serializable {
	private static final long serialVersionUID = 64565445665456666L;

	@Id
	@Column(name="ID")
	@GeneratedValue
	private long id;

	private int quantity;

	private String productId;

	public OrderItem() {}

	public String getProductId() {
		return productId;
	}

	public void setProductId(String productId) {
		this.productId = productId;
	}

	public int getQuantity() {
		return quantity;
	}

	public void setQuantity(int quantity) {
		this.quantity = quantity;
	}

	@Override
	public String toString() {
		return "OrderItem [productId=" + productId + ", quantity=" + quantity + "]";
	}

}
```

## Issues

### incident 0
incident to fix: "The way in which Hibernate determines implicit names for sequences and tables associated with identifier generation has changed in 6.0 which may affect migrating applications. 
 As of 6.0, Hibernate by default creates a sequence per entity hierarchy instead of a single sequence hibernate_sequence. 
 Due to this change, users that previously used `@GeneratedValue(strategy = GenerationStrategy.AUTO)` or simply `@GeneratedValue` (since `AUTO` is the default), need to ensure that the database now contains sequences for every entity, named `<entity name>_seq`. For an entity Person, a sequence person_seq is expected to exist. 
 It’s best to run hbm2ddl (e.g. by temporarily setting `hbm2ddl.auto=create`) to obtain a list of DDL statements for the sequences."
Line number: 18
### incident 1
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 5
### incident 2
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 6
### incident 3
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 7
### incident 4
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 8
### incident 5
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 9

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.

''',
    4: '''
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/service/ProductService.java"
Source file contents:
```java
package com.redhat.coolstore.service;

import com.redhat.coolstore.model.CatalogItemEntity;
import com.redhat.coolstore.model.Product;
import com.redhat.coolstore.utils.Transformers;

import javax.ejb.Stateless;
import javax.inject.Inject;
import java.util.List;
import java.util.stream.Collectors;

import static com.redhat.coolstore.utils.Transformers.toProduct;

@Stateless
public class ProductService {

    @Inject
    CatalogService cm;

    public ProductService() {
    }

    public List<Product> getProducts() {
        return cm.getCatalogItems().stream().map(entity -> toProduct(entity)).collect(Collectors.toList());
    }

    public Product getProductByItemId(String itemId) {
        CatalogItemEntity entity = cm.getCatalogItemById(itemId);
        if (entity == null)
            return null;

        // Return the entity
        return Transformers.toProduct(entity);
    }

}

```

## Issues

### incident 0
incident to fix: "Replace the `javax.ejb` import statement with `jakarta.ejb`"
Line number: 7
### incident 1
incident to fix: "Replace the `javax.inject` import statement with `jakarta.inject`"
Line number: 8
### incident 2
incident to fix: "Stateless EJBs can be converted to a CDI bean by replacing the `@Stateless` annotation with a scope eg `@ApplicationScoped`"
Line number: 14

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.

''',
    5: '''
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/service/ShoppingCartService.java"
Source file contents:
```java
package com.redhat.coolstore.service;

import java.util.Hashtable;
import java.util.logging.Logger;

import javax.ejb.Stateful;
import javax.inject.Inject;
import javax.naming.Context;
import javax.naming.InitialContext;
import javax.naming.NamingException;

import com.redhat.coolstore.model.Product;
import com.redhat.coolstore.model.ShoppingCart;
import com.redhat.coolstore.model.ShoppingCartItem;

@Stateful
public class ShoppingCartService  {

    @Inject
    Logger log;

    @Inject
    ProductService productServices;

    @Inject
    PromoService ps;


    @Inject
    ShoppingCartOrderProcessor shoppingCartOrderProcessor;

    private ShoppingCart cart  = new ShoppingCart(); //Each user can have multiple shopping carts (tabbed browsing)

   

    public ShoppingCartService() {
    }

    public ShoppingCart getShoppingCart(String cartId) {
        return cart;
    }

    public ShoppingCart checkOutShoppingCart(String cartId) {
        ShoppingCart cart = this.getShoppingCart(cartId);
      
        log.info("Sending  order: ");
        shoppingCartOrderProcessor.process(cart);
   
        cart.resetShoppingCartItemList();
        priceShoppingCart(cart);
        return cart;
    }

    public void priceShoppingCart(ShoppingCart sc) {

        if (sc != null) {

            initShoppingCartForPricing(sc);

            if (sc.getShoppingCartItemList() != null && sc.getShoppingCartItemList().size() > 0) {

                ps.applyCartItemPromotions(sc);

                for (ShoppingCartItem sci : sc.getShoppingCartItemList()) {

                    sc.setCartItemPromoSavings(
                            sc.getCartItemPromoSavings() + sci.getPromoSavings() * sci.getQuantity());
                    sc.setCartItemTotal(sc.getCartItemTotal() + sci.getPrice() * sci.getQuantity());

                }

                sc.setShippingTotal(lookupShippingServiceRemote().calculateShipping(sc));

                if (sc.getCartItemTotal() >= 25) {
                    sc.setShippingTotal(sc.getShippingTotal()
                            + lookupShippingServiceRemote().calculateShippingInsurance(sc));
                }

            }

            ps.applyShippingPromotions(sc);

            sc.setCartTotal(sc.getCartItemTotal() + sc.getShippingTotal());

        }

    }

    private void initShoppingCartForPricing(ShoppingCart sc) {

        sc.setCartItemTotal(0);
        sc.setCartItemPromoSavings(0);
        sc.setShippingTotal(0);
        sc.setShippingPromoSavings(0);
        sc.setCartTotal(0);

        for (ShoppingCartItem sci : sc.getShoppingCartItemList()) {
            Product p = getProduct(sci.getProduct().getItemId());
            //if product exist
            if (p != null) {
                sci.setProduct(p);
                sci.setPrice(p.getPrice());
            }

            sci.setPromoSavings(0);
        }

    }

    public Product getProduct(String itemId) {
        return productServices.getProductByItemId(itemId);
    }

	private static ShippingServiceRemote lookupShippingServiceRemote() {
        try {
            final Hashtable<String, String> jndiProperties = new Hashtable<>();
            jndiProperties.put(Context.INITIAL_CONTEXT_FACTORY, "org.wildfly.naming.client.WildFlyInitialContextFactory");

            final Context context = new InitialContext(jndiProperties);

            return (ShippingServiceRemote) context.lookup("ejb:/ROOT/ShippingService!" + ShippingServiceRemote.class.getName());
        } catch (NamingException e) {
            throw new RuntimeException(e);
        }
    }
}

```

## Issues

### incident 0
incident to fix: "Replace the `javax.ejb` import statement with `jakarta.ejb`"
Line number: 6
### incident 1
incident to fix: "Replace the `javax.inject` import statement with `jakarta.inject`"
Line number: 7
### incident 2
incident to fix: "Stateful EJBs can be converted to a CDI bean by replacing the `@Stateful` annotation with a bean-defining annotation
 that encompasses the appropriate scope (e.g., `@ApplicationScoped`). `@Stateful` EJBs often translate to `@SessionScoped`
 beans (a scope which requires activating the `quarkus-undertow` extension), but the appropriate scope may differ based
 on your application architecture. Review your application's requirements to determine the appropriate scope.

 Note that it is recommended, as a good practice, to keep state external from the service in Quarkus."
Line number: 16

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.

''',
    6: '''
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/service/ShoppingCartOrderProcessor.java"
Source file contents:
```java
package com.redhat.coolstore.service;

import java.util.logging.Logger;
import javax.ejb.Stateless;
import javax.annotation.Resource;
import javax.inject.Inject;
import javax.jms.JMSContext;
import javax.jms.Topic;

import com.redhat.coolstore.model.ShoppingCart;
import com.redhat.coolstore.utils.Transformers;

@Stateless
public class ShoppingCartOrderProcessor  {

    @Inject
    Logger log;


    @Inject
    private transient JMSContext context;

    @Resource(lookup = "java:/topic/orders")
    private Topic ordersTopic;

    
  
    public void  process(ShoppingCart cart) {
        log.info("Sending order from processor: ");
        context.createProducer().send(ordersTopic, Transformers.shoppingCartToJson(cart));
    }



}

```

## Issues

### incident 0
incident to fix: "Replace the `javax.annotation` import statement with `jakarta.annotation`"
Line number: 5
### incident 1
incident to fix: "Replace the `javax.ejb` import statement with `jakarta.ejb`"
Line number: 4
### incident 2
incident to fix: "Replace the `javax.inject` import statement with `jakarta.inject`"
Line number: 6
### incident 3
incident to fix: "Replace the `javax.jms` import statement with `jakarta.jms`"
Line number: 7
### incident 4
incident to fix: "Replace the `javax.jms` import statement with `jakarta.jms`"
Line number: 8
### incident 5
incident to fix: "Stateless EJBs can be converted to a CDI bean by replacing the `@Stateless` annotation with a scope eg `@ApplicationScoped`"
Line number: 13
### incident 6
incident to fix: "JMS `Topic`s should be replaced with Micrometer `Emitter`s feeding a Channel. See the following example of migrating
 a Topic to an Emitter:
 
 Before:
 ```
 @Resource(lookup = "java:/topic/HELLOWORLDMDBTopic")
 private Topic topic;
 ```
 
 After:
 ```
 @Inject
 @Channel("HELLOWORLDMDBTopic")
 Emitter<String> topicEmitter;
 ```"
Line number: 8
### incident 7
incident to fix: "JMS `Topic`s should be replaced with Micrometer `Emitter`s feeding a Channel. See the following example of migrating
 a Topic to an Emitter:
 
 Before:
 ```
 @Resource(lookup = "java:/topic/HELLOWORLDMDBTopic")
 private Topic topic;
 ```
 
 After:
 ```
 @Inject
 @Channel("HELLOWORLDMDBTopic")
 Emitter<String> topicEmitter;
 ```"
Line number: 24
### incident 8
incident to fix: "References to JavaEE/JakartaEE JMS elements should be removed and replaced with their Quarkus SmallRye/Microprofile equivalents."
Line number: 7
### incident 9
incident to fix: "References to JavaEE/JakartaEE JMS elements should be removed and replaced with their Quarkus SmallRye/Microprofile equivalents."
Line number: 8

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.

''',
    7: '''
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/utils/DataBaseMigrationStartup.java"
Source file contents:
```java
package com.redhat.coolstore.utils;

import org.flywaydb.core.Flyway;
import org.flywaydb.core.api.FlywayException;

import javax.annotation.PostConstruct;
import javax.annotation.Resource;
import javax.ejb.Singleton;
import javax.ejb.Startup;
import javax.ejb.TransactionManagement;
import javax.ejb.TransactionManagementType;
import javax.inject.Inject;
import javax.sql.DataSource;
import java.util.logging.Level;
import java.util.logging.Logger;

/**
 * Created by tqvarnst on 2017-04-04.
 */
@Singleton
@Startup
@TransactionManagement(TransactionManagementType.BEAN)
public class DataBaseMigrationStartup {

    @Inject
    Logger logger;

    @Resource(mappedName = "java:jboss/datasources/CoolstoreDS")
    DataSource dataSource;

    @PostConstruct
    private void startup() {


        try {
            logger.info("Initializing/migrating the database using FlyWay");
            Flyway flyway = new Flyway();
            flyway.setDataSource(dataSource);
            flyway.baseline();
            // Start the db.migration
            flyway.migrate();
        } catch (FlywayException e) {
            if(logger !=null)
                logger.log(Level.SEVERE,"FAILED TO INITIALIZE THE DATABASE: " + e.getMessage(),e);
            else
                System.out.println("FAILED TO INITIALIZE THE DATABASE: " + e.getMessage() + " and injection of logger doesn't work");

        }
    }



}
```

## Issues

### incident 0
incident to fix: "Replace the `javax.annotation` import statement with `jakarta.annotation`"
Line number: 6
### incident 1
incident to fix: "Replace the `javax.annotation` import statement with `jakarta.annotation`"
Line number: 7
### incident 2
incident to fix: "Replace the `javax.ejb` import statement with `jakarta.ejb`"
Line number: 8
### incident 3
incident to fix: "Replace the `javax.ejb` import statement with `jakarta.ejb`"
Line number: 9
### incident 4
incident to fix: "Replace the `javax.ejb` import statement with `jakarta.ejb`"
Line number: 10
### incident 5
incident to fix: "Replace the `javax.ejb` import statement with `jakarta.ejb`"
Line number: 11
### incident 6
incident to fix: "Replace the `javax.inject` import statement with `jakarta.inject`"
Line number: 12

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.

''',
    8: '''
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/service/PromoService.java"
Source file contents:
```java
package com.redhat.coolstore.service;

import java.io.Serializable;
import java.util.HashMap;
import java.util.HashSet;
import java.util.Map;
import java.util.Set;

import javax.enterprise.context.ApplicationScoped;

import com.redhat.coolstore.model.Promotion;
import com.redhat.coolstore.model.ShoppingCart;
import com.redhat.coolstore.model.ShoppingCartItem;

@ApplicationScoped
public class PromoService implements Serializable {

    private static final long serialVersionUID = 2088590587856645568L;

    private String name = null;

    private Set<Promotion> promotionSet = null;

    public PromoService() {

        promotionSet = new HashSet<>();

        promotionSet.add(new Promotion("329299", .25));

    }

    public void applyCartItemPromotions(ShoppingCart shoppingCart) {

        if (shoppingCart != null && shoppingCart.getShoppingCartItemList().size() > 0) {

            Map<String, Promotion> promoMap = new HashMap<String, Promotion>();

            for (Promotion promo : getPromotions()) {

                promoMap.put(promo.getItemId(), promo);

            }

            for (ShoppingCartItem sci : shoppingCart.getShoppingCartItemList()) {

                String productId = sci.getProduct().getItemId();

                Promotion promo = promoMap.get(productId);

                if (promo != null) {

                    sci.setPromoSavings(sci.getProduct().getPrice() * promo.getPercentOff() * -1);
                    sci.setPrice(sci.getProduct().getPrice() * (1 - promo.getPercentOff()));

                }

            }

        }

    }

    public void applyShippingPromotions(ShoppingCart shoppingCart) {

        if (shoppingCart != null) {

            //PROMO: if cart total is greater than 75, free shipping
            if (shoppingCart.getCartItemTotal() >= 75) {

                shoppingCart.setShippingPromoSavings(shoppingCart.getShippingTotal() * -1);
                shoppingCart.setShippingTotal(0);

            }

        }

    }

    public Set<Promotion> getPromotions() {

        if (promotionSet == null) {

            promotionSet = new HashSet<>();

        }

        return new HashSet<>(promotionSet);

    }

    public void setPromotions(Set<Promotion> promotionSet) {

        if (promotionSet != null) {

            this.promotionSet = new HashSet<>(promotionSet);

        } else {

            this.promotionSet = new HashSet<>();

        }

    }

    @Override
    public String toString() {
        return "PromoService [name=" + name + ", promotionSet=" + promotionSet + "]";
    }

}

```

## Issues

### incident 0
incident to fix: "Replace the `javax.enterprise` import statement with `jakarta.enterprise`"
Line number: 9

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.

''',
    9: '''
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/persistence/Resources.java"
Source file contents:
```java
package com.redhat.coolstore.persistence;

import javax.enterprise.context.Dependent;
import javax.enterprise.inject.Produces;
import javax.persistence.EntityManager;
import javax.persistence.PersistenceContext;

@Dependent
public class Resources {

    @PersistenceContext
    private EntityManager em;

    @Produces
    public EntityManager getEntityManager() {
        return em;
    }
}

```

## Issues

### incident 0
incident to fix: "Replace the `javax.enterprise` import statement with `jakarta.enterprise`"
Line number: 3
### incident 1
incident to fix: "Replace the `javax.enterprise` import statement with `jakarta.enterprise`"
Line number: 4
### incident 2
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 5
### incident 3
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 6
### incident 4
incident to fix: "In Quarkus, you can skip the @Produces annotation completely if the producer method is annotated with a scope annotation, a stereotype or a qualifier..
 This field could be accessed using a `@Named` getter method instead."
Line number: 14
### incident 5
incident to fix: "In JavaEE/JakartaEE, using `@PersistenceContext` was needed in order to inject a data source. Quarkus, on the other hand,
 will create the bean automatically just by correctly setting up your datasource, so the `@PersistenceContext` annotation can be removed. 
This also makes having a `@Produces` annotation
 on the `EntityManager` illegal in Quarkus.
 
 If you are using a `@Produces` annotation for your EntityManager, and it is not needed after configuring your datasource, remove it and `@Inject` the EntityManager.
 Otherwise, if the producer is still needed, please create a qualification for your produced `EntityManager`, as well as every injection point for the EM.
 
 For instance, you can create an `ExtendedContext` qualifier:
 ```
 @Qualifier
 @Target()
 @Retention(RetentionPolicy.RUNTIME)
 public @interface ExtendedContext 
 ```
 and then inject your entity managers:
 ```
 @ExtendedContext
 public EntityManager getEm() 
 ```"
Line number: 5

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.

''',
    10: '''
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/rest/OrderEndpoint.java"
Source file contents:
```java
package com.redhat.coolstore.rest;

import java.io.Serializable;
import java.util.List;

import javax.enterprise.context.RequestScoped;
import javax.inject.Inject;
import javax.ws.rs.Consumes;
import javax.ws.rs.GET;
import javax.ws.rs.Path;
import javax.ws.rs.PathParam;
import javax.ws.rs.Produces;
import javax.ws.rs.core.MediaType;

import com.redhat.coolstore.model.Order;
import com.redhat.coolstore.service.OrderService;

@RequestScoped
@Path("/orders")
@Consumes(MediaType.APPLICATION_JSON)
@Produces(MediaType.APPLICATION_JSON)
public class OrderEndpoint implements Serializable {

    private static final long serialVersionUID = -7227732980791688774L;

    @Inject
    private OrderService os;


    @GET
    @Path("/")
    public List<Order> listAll() {
        return os.getOrders();
    }

    @GET
    @Path("/{orderId}")
    public Order getOrder(@PathParam("orderId") long orderId) {
        return os.getOrderById(orderId);
    }

}

```

## Issues

### incident 0
incident to fix: "Replace the `javax.enterprise` import statement with `jakarta.enterprise`"
Line number: 6
### incident 1
incident to fix: "Replace the `javax.inject` import statement with `jakarta.inject`"
Line number: 7
### incident 2
incident to fix: "Replace the `javax.ws` import statement with `jakarta.ws`"
Line number: 8
### incident 3
incident to fix: "Replace the `javax.ws` import statement with `jakarta.ws`"
Line number: 9
### incident 4
incident to fix: "Replace the `javax.ws` import statement with `jakarta.ws`"
Line number: 10
### incident 5
incident to fix: "Replace the `javax.ws` import statement with `jakarta.ws`"
Line number: 11
### incident 6
incident to fix: "Replace the `javax.ws` import statement with `jakarta.ws`"
Line number: 12
### incident 7
incident to fix: "Replace the `javax.ws` import statement with `jakarta.ws`"
Line number: 13

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.

'''
}


In [4]:
def estimating_tokens(prompt):
    enc = tiktoken.encoding_for_model("gpt-4o")
    estimate_prompt_tokens = len(enc.encode(prompt))
    return int(estimate_prompt_tokens)

In [5]:
def actual_tokens_func(prompt):
    client = OpenAI(
        api_key=os.getenv("OPENAI_API_KEY")
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role":"user",
                "content":prompt
            }
        ]
    )
    # conversion to json
    json_response = json.loads(response.model_dump_json())
    actual_token_info = json_response['usage']['prompt_tokens']
    return int(actual_token_info)


In [6]:
percentage_off_list =[]

In [7]:

for prompt_number,prompt in (prompts.items()):
    print(f"Estimating tokens {estimating_tokens(prompt)} in prompt number {prompt_number}")
    print(f"Actual tokens {actual_tokens_func(prompt)} in prompt number {prompt_number}")

    estimated_tokens = estimating_tokens(prompt)
    actual_tokens = actual_tokens_func(prompt)

    difference = int(actual_tokens) - int(estimated_tokens)
    percentage_off = int((int(difference) / int(estimated_tokens)) * 100)
    print(percentage_off)
    percentage_off_list.append(percentage_off)

print(percentage_off_list)




    

Estimating tokens 859 in prompt number 1
Actual tokens 866 in prompt number 1
0
Estimating tokens 1052 in prompt number 2
Actual tokens 1059 in prompt number 2
0
Estimating tokens 896 in prompt number 3
Actual tokens 903 in prompt number 3
0
Estimating tokens 660 in prompt number 4
Actual tokens 667 in prompt number 4
1
Estimating tokens 1298 in prompt number 5
Actual tokens 1305 in prompt number 5
0
Estimating tokens 1014 in prompt number 6
Actual tokens 1021 in prompt number 6
0
Estimating tokens 882 in prompt number 7
Actual tokens 889 in prompt number 7
0
Estimating tokens 944 in prompt number 8
Actual tokens 951 in prompt number 8
0
Estimating tokens 842 in prompt number 9
Actual tokens 849 in prompt number 9
0
Estimating tokens 826 in prompt number 10
Actual tokens 833 in prompt number 10
0
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]


In [8]:
def average_percentage():
    added = 0
    for i in percentage_off_list:
        added += i
    return(added/len(percentage_off_list))


After the results, we can see that we have an average of 0.1% for the estimation.

In [9]:
print(f"AVERAGE PERCENTAGE DIFFERENCE: {average_percentage()}%")

AVERAGE PERCENTAGE DIFFERENCE: 0.1%
